### Imports

In [ ]:
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, utils
from torch.utils.data import TensorDataset, DataLoader
from pathlib import Path
import os
import matplotlib.pyplot as plt


from art.attacks.evasion import ElasticNet
from art.attacks.evasion import ProjectedGradientDescentPyTorch as PGDAttack
from art.estimators.classification import PyTorchClassifier
from art.utils import load_mnist

from _utils import test

%matplotlib inline
%config InlineBackend.figure_format='retina'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_cuda = True

In [ ]:
(x_train, y_train), (x_test, y_test), min_, max_ = load_mnist()

x_train = np.swapaxes(x_train, 1, 3).astype(np.float32)
x_test = np.swapaxes(x_test, 1, 3).astype(np.float32)

train_dataset = TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
train_dataloader = DataLoader(train_dataset, batch_size=128, num_workers=6)

test_dataset = TensorDataset(torch.Tensor(x_test), torch.Tensor(y_test))
test_dataloader = DataLoader(test_dataset, batch_size=1000)
test_dataloader_single =  DataLoader(test_dataset, batch_size=1, num_workers=6)

In [ ]:
pretrained_model = "MNIST_vanilla_model_01.pth"
use_cuda = True

### Classifier

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 512)
        self.fc2 = nn.Linear(512, 200)
        self.fc3 = nn.Linear(200,10)
    
    activations_fc2 = []
    mask_fc2 = torch.zeros(1, 200)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training)
        x = self.fc3(x)
        return x
    
    def forwardDetect(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        self.activations_fc2.append(x)
        x = self.fc3(x)
        return x
    
    def forwardMask(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))*self.mask_fc2
        x = self.fc3(x)
        return x
    
# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

# Initialize the model.
model = Classifier().to(device)

# # Load pre-trained model
model.load_state_dict(torch.load(pretrained_model, map_location='cpu'))

# # Set the model in evaluation mode. In this case this is for the Dropout layers
print(model.eval())

# Load loss and optimiser
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

# Make a classifier wrapper!
classifier = PyTorchClassifier(
    model=model,
    clip_values=(min_, max_),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(1, 28, 28),
    nb_classes=10,
)

# Test model
predictions = classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {} %".format(accuracy * 100))

### Mask Classifier

In [ ]:
class MaskModel(nn.Module):
    def __init__(self):
        super(MaskModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 512)
        self.fc2 = nn.Linear(512, 200)
        self.fc3 = nn.Linear(200,10)
    
    mask_fc2 = torch.ones(1, 200)
        
    
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))*self.mask_fc2.to(device)
        x = self.fc3(x)
        return x
    
# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

# Initialize the model.
mask_model = MaskModel().to(device)

# Load pre-trained model
mask_model.load_state_dict(torch.load(pretrained_model, map_location='cpu'))

# Set the model in evaluation mode. In this case this is for the Dropout layers
print(mask_model.eval())

# Load loss and optimiser
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(mask_model.parameters(), lr=0.01, momentum=0.5)

# Make a classifier wrapper!
mask_classifier = PyTorchClassifier(
    model=mask_model,
    clip_values=(min_, max_),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(1, 28, 28),
    nb_classes=10,
)

# Test model
predictions = mask_classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {} %".format(accuracy * 100))

### Pruning

In [ ]:
model.eval() 
with torch.no_grad():
    for data, target in test_dataloader_single:
        output = model.forwardDetect(data.to(device))

activations_fc2 = model.activations_fc2

In [ ]:
n = len(activations_fc2)
element0 = activations_fc2.pop()
act = torch.zeros(n,element0.size(0),element0.size(1))
act[0,:,:] = element0

for e in range(n-1):
    elementX = activations_fc2.pop()
    act[e+1,:,:] = elementX

mean_activations_fc2 = torch.mean(act,dim=[0,1])
ma = mean_activations_fc2.numpy()
plt.hist(ma,80)
plt.title('Activations')
plt.show()

In [ ]:
# save different masks for different percentages

masks_fc2 = dict()

for percentage in [0, 3, 6, 9]:
    s_ma_fc2, idx_ma_fc2 = torch.sort(mean_activations_fc2)
    m_fc2 = torch.ones(mean_activations_fc2.size()).to(device)
    nn_fc2 = mean_activations_fc2.size(0)
    ind_r_fc2 = round((percentage/10)*nn_fc2)
    m_fc2[idx_ma_fc2[:ind_r_fc2]] = 0.0
    n1_fc2 = m_fc2.sum()
    model.mask_fc2 = m_fc2
    masks_fc2[percentage/10] = model.mask_fc2

In [ ]:
# test accuracy on pruned network
for (pg, mask) in masks_fc2.items():
    print('\n Pruned {}'.format(pg))
    mask_model.mask_fc2 = mask
    predictions = mask_classifier.predict(x_test)
    accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
    print("Accuracy on benign test examples: {} %".format(accuracy * 100))

### Elastic Net attack

In [ ]:
def EAD_L1(classifier, x_test):
    attack = ElasticNet(classifier=classifier,
                        learning_rate=1e-2,
                        binary_search_steps=5,
                        max_iter=10,
                        beta=1e-3,
                        initial_const=1e-3,
                        decision_rule="L1")
    x_test_adv = attack.generate(x=x_test)
    return x_test_adv

In [ ]:
# create adversarial samples on pruned models and attack pruned models 

n = 1000
for (pg, mask) in masks_fc2.items():
    print('\n Pruned {}'.format(pg))
    mask_model.mask_fc2 = mask
    x_test_adv_l1 = EAD_L1(mask_classifier, x_test[:n])
    predictions = mask_classifier.predict(x_test_adv_l1)
    acc = 100*(np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[:n], axis=1)) / n)
    
    print(' Accuracy:', acc)

### Projected Gradient Descent attack

In [ ]:
def PGD(classifier, x_test, norm, eps):
    attack = PGDAttack(estimator=classifier,
                       norm=norm,
                       eps=eps,
                       eps_step=0.1,
                       max_iter=100,
                       verbose=False)
    x_test_adv = attack.generate(x=x_test)
    return x_test_adv

In [ ]:
# L2
n = 1000
for (pg, mask) in masks_fc2.items():
    print('\n Pruned {}'.format(pg))
    mask_model.mask_fc2 = mask
    x_test_adv_l2 = PGD(mask_classifier, x_test[:n], norm=2, eps=2)
    predictions = mask_classifier.predict(x_test_adv_l2)
    acc = 100*(np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[:n], axis=1)) / n)
    
    print(' Accuracy:', acc)

In [ ]:
# Linf
n = 1000
for (pg, mask) in masks_fc2.items():
    print('\n Pruned {}'.format(pg))
    mask_model.mask_fc2 = mask
    x_test_adv_linf = PGD(mask_classifier, x_test[:n], norm=np.inf, eps=.3)
    predictions = mask_classifier.predict(x_test_adv_linf)
    acc = 100*(np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[:n], axis=1)) / n)
    
    print(' Accuracy:', acc)